# Flight Prediction Web Scraping 

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import selenium.webdriver
from selenium import webdriver 
from datetime import date 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
from dateutil.relativedelta import relativedelta
import csv
import linecache
import sys

In [2]:
def PrintException():
    exc_type,exc_obj,tb = sys.exc_info()
    f= tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno,f.f_globals)
    print ('Exception in ({},Line {} "{}"):{}'.format(filename, lineno,line.strip, exc_obj))

In [3]:
def scrap (origin, destin,trDate):
    global none
    baseDataUrl = "https://www.makemytrip.com/flight/search?itinerary="+ origin +"-"+ destin +"-"+ trDate +"&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|"
    try:
        driver = webdriver.Chrome(r"C:\Users\nikit\OneDrive\Desktop\DS Projects\WS Selenium\chromedriver.exe")
        print("Requested Url: "+baseDataUrl)
        
        driver.get(baseDataUrl)
        print("WebPage found...")
        
        element_xpath = '//*[@id="left-side--wrapper"]/div[2]'
        
        print("Scrolling document upto bottom...")
        for j in range(1, 100):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        
        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")

        print("Closing Chrome ...")
        driver.quit()

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body)
        
        spanFlightName = soupBody.find_all("span", 'boldFont blackText airlineName')
        print("spanFlightName  = " + str(len(spanFlightName)))

        Time = soupBody.find_all("p", {"class":"blackText fontSize18 blackFont appendBottom2 makeFlex hrtlCenter"})			# Tags with Departure Time
        print("Time Count in Scraping  = " + str(len(Time)))

        City = soupBody.findAll("p", {"class": "darkText"})
        print("City Count in Scraping  = " + str(len(City)))

        spanFlightCost = soupBody.findAll("p", {"class": "blackText fontSize18 blackFont white-space-no-wrap"})			# Tags with Flight Cost
        print("spanFlightCost Count in Scraping  = " + str(len(spanFlightCost)))

        pFlightStops = soupBody.findAll("p", {"class": "flightsLayoverInfo"})
        print("pFlightStops Count in Scraping  = " + str(len(pFlightStops)))
        
        transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops)



   
    except :
        PrintException()
        print(trDate," scraping complete")
        
        none =True 

In [5]:
dates=(pd.date_range(start='25/01/2022', end='31/01/2022'))
New_date=[]
for i,date in enumerate(dates):
    day=str(date.day).zfill(2)   
    month=str(date.month).zfill(2)
    year=str(date.year)
    date=day+'/'+month+'/'+year  
    New_date.append(date)
print(New_date)

['25/01/2022', '26/01/2022', '27/01/2022', '28/01/2022', '29/01/2022', '30/01/2022', '31/01/2022']


In [6]:
#Lists to store scraped data
Airline=[]
Journey_date=[]
From=[]
To=[]
Price=[]
Dtime=[]
Atime=[]
Stops=[]

flight_data = pd.DataFrame(np.nan, index=[0], columns=['Airline', 'Journey_date','From','To','Dtime','Atime','Stops','Price'])
flight_data.to_csv('Flight.csv')
 
def transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops):
    
    Airline.clear()
    Journey_date.clear()
    Price.clear()
    Stops.clear()
    From.clear()
    Dtime.clear()
    To.clear()
    Atime.clear()
    
    global flight_data
    print("Transaform Function Running.... ")
    for i in range(0,len(spanFlightName)):
        
        Airline.append(spanFlightName[i].string)
        Journey_date.append(trDate)
        Price.append(spanFlightCost[i].string)
        Stops.append(pFlightStops[i].string)
        
      
    for j in range(0,len(Time)):
        
        if(j%2==0):
            
            From.append(City[j].string)
            Dtime.append(Time[j].string)
        else:
            
            To.append(City[j].string)
            Atime.append(Time[j].string) 

            
            
    temp = {'Airline':Airline,
            'Journey_date':Journey_date,
            'From':From,
            'To':To,
            'Dtime':Dtime,
            'Atime':Atime,
            'Stops':Stops,
            'Price':Price}
    df=pd.DataFrame(temp)
    df.to_csv('Flight.csv', mode='a', header=False)
    flight_data = pd.concat([flight_data,df])
    flight_data=flight_data.reset_index(drop=True)
    display(flight_data)
    print('Lenght : ',len(flight_data))

In [7]:
list_of_dest=['GOI','AMD','TIR','BLR','BOM','DEL']
for i in range(0,len(list_of_dest)):
    orgin=list_of_dest[i]
    for_to=list_of_dest
    print("--------------")
    for j in range(0,len(for_to)):
        destin=for_to[j]
        none = False
        print("origin = "+orgin)
        print("destin = "+destin+'\n')
        if orgin!=destin:
            for i,date in enumerate(New_date) :
                if none !=True:
                    scrap(orgin,destin,date)
                    print(" ")
                else:
                    break
            else: 
                print( "-----Skiped-----")
                print(" ")

--------------
origin = GOI
destin = GOI

origin = GOI
destin = AMD

Requested Url: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-25/01/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
WebPage found...
Scrolling document upto bottom...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 15
Time Count in Scraping  = 30
City Count in Scraping  = 30
spanFlightCost Count in Scraping  = 15
pFlightStops Count in Scraping  = 15
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,25/01/2022,Goa,Ahmedabad,06:20,08:20,Non stop,"₹ 5,315"
2,Go First,25/01/2022,Goa,Ahmedabad,13:15,20:25,1 stop via Mumbai,"₹ 5,315"
3,Go First,25/01/2022,Goa,Ahmedabad,14:00,20:55,1 stop via Bengaluru,"₹ 5,315"
4,Go First,25/01/2022,Goa,Ahmedabad,23:55,06:40,1 stop via Mumbai,"₹ 5,315"
5,Go First,25/01/2022,Goa,Ahmedabad,18:15,06:40,1 stop via Mumbai,"₹ 5,315"
6,Go First,25/01/2022,Goa,Ahmedabad,17:00,06:40,1 stop via Mumbai,"₹ 5,315"
7,Spicejet,25/01/2022,Goa,Ahmedabad,13:55,08:20,1 stop via Pune,"₹ 5,315"
8,IndiGo,25/01/2022,Goa,Ahmedabad,13:10,18:30,1 stop via Mumbai,"₹ 5,316"
9,IndiGo,25/01/2022,Goa,Ahmedabad,15:25,20:45,1 stop via Bengaluru,"₹ 5,316"


Lenght :  16
 
Requested Url: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-26/01/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
WebPage found...
Scrolling document upto bottom...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 14
Time Count in Scraping  = 28
City Count in Scraping  = 30
spanFlightCost Count in Scraping  = 14
pFlightStops Count in Scraping  = 14
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,25/01/2022,Goa,Ahmedabad,06:20,08:20,Non stop,"₹ 5,315"
2,Go First,25/01/2022,Goa,Ahmedabad,13:15,20:25,1 stop via Mumbai,"₹ 5,315"
3,Go First,25/01/2022,Goa,Ahmedabad,14:00,20:55,1 stop via Bengaluru,"₹ 5,315"
4,Go First,25/01/2022,Goa,Ahmedabad,23:55,06:40,1 stop via Mumbai,"₹ 5,315"
5,Go First,25/01/2022,Goa,Ahmedabad,18:15,06:40,1 stop via Mumbai,"₹ 5,315"
6,Go First,25/01/2022,Goa,Ahmedabad,17:00,06:40,1 stop via Mumbai,"₹ 5,315"
7,Spicejet,25/01/2022,Goa,Ahmedabad,13:55,08:20,1 stop via Pune,"₹ 5,315"
8,IndiGo,25/01/2022,Goa,Ahmedabad,13:10,18:30,1 stop via Mumbai,"₹ 5,316"
9,IndiGo,25/01/2022,Goa,Ahmedabad,15:25,20:45,1 stop via Bengaluru,"₹ 5,316"


Lenght :  30
 
Requested Url: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-27/01/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
WebPage found...
Scrolling document upto bottom...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 13
Time Count in Scraping  = 26
City Count in Scraping  = 28
spanFlightCost Count in Scraping  = 13
pFlightStops Count in Scraping  = 13
Transaform Function Running.... 


...

Lenght :  1514
 
Requested Url: https://www.makemytrip.com/flight/search?itinerary=BLR-TIR-31/01/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
Exception in (<ipython-input-3-c12bd3f3d540>,Line 8 "<built-in method strip of str object at 0x0000019DCFE31A50>"):Message: timeout: Timed out receiving message from renderer: 299.638
  (Session info: chrome=97.0.4692.71)

31/01/2022  scraping complete
 
-----Skiped-----
 
origin = BLR
destin = BLR

origin = BLR
destin = BOM

Requested Url: https://www.makemytrip.com/flight/search?itinerary=BLR-BOM-25/01/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
WebPage found...
Scrolling document upto bottom...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 20
Time Count in Scraping  = 40
City Count in Scraping  = 40
spanFlightCo

,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,25/01/2022,Goa,Ahmedabad,06:20,08:20,Non stop,"₹ 5,315"
2,Go First,25/01/2022,Goa,Ahmedabad,13:15,20:25,1 stop via Mumbai,"₹ 5,315"
3,Go First,25/01/2022,Goa,Ahmedabad,14:00,20:55,1 stop via Bengaluru,"₹ 5,315"
4,Go First,25/01/2022,Goa,Ahmedabad,23:55,06:40,1 stop via Mumbai,"₹ 5,315"
...,...,...,...,...,...,...,...,...
1529,Air India,25/01/2022,Bengaluru,Mumbai,06:45,08:35,Non stop,"₹ 5,179"
1530,Air India,25/01/2022,Bengaluru,Mumbai,19:10,21:05,Non stop,"₹ 5,179"
1531,Vistara,25/01/2022,Bengaluru,Mumbai,09:30,11:25,Non stop,"₹ 5,180"
1532,Vistara,25/01/2022,Bengaluru,Mumbai,19:55,21:40,Non stop,"₹ 5,757"


Lenght :  1534
 
Requested Url: https://www.makemytrip.com/flight/search?itinerary=BLR-BOM-26/01/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
WebPage found...
Scrolling document upto bottom...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 0
Time Count in Scraping  = 0
City Count in Scraping  = 0
spanFlightCost Count in Scraping  = 0
pFlightStops Count in Scraping  = 0
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,25/01/2022,Goa,Ahmedabad,06:20,08:20,Non stop,"₹ 5,315"
2,Go First,25/01/2022,Goa,Ahmedabad,13:15,20:25,1 stop via Mumbai,"₹ 5,315"
3,Go First,25/01/2022,Goa,Ahmedabad,14:00,20:55,1 stop via Bengaluru,"₹ 5,315"
4,Go First,25/01/2022,Goa,Ahmedabad,23:55,06:40,1 stop via Mumbai,"₹ 5,315"
...,...,...,...,...,...,...,...,...
1529,Air India,25/01/2022,Bengaluru,Mumbai,06:45,08:35,Non stop,"₹ 5,179"
1530,Air India,25/01/2022,Bengaluru,Mumbai,19:10,21:05,Non stop,"₹ 5,179"
1531,Vistara,25/01/2022,Bengaluru,Mumbai,09:30,11:25,Non stop,"₹ 5,180"
1532,Vistara,25/01/2022,Bengaluru,Mumbai,19:55,21:40,Non stop,"₹ 5,757"


Lenght :  1534
 
Requested Url: https://www.makemytrip.com/flight/search?itinerary=BLR-BOM-27/01/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
Exception in (<ipython-input-3-c12bd3f3d540>,Line 8 "<built-in method strip of str object at 0x0000019DCFE31A50>"):Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=97.0.4692.71)

27/01/2022  scraping complete
 
origin = BLR
destin = DEL

Requested Url: https://www.makemytrip.com/flight/search?itinerary=BLR-DEL-25/01/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
Exception in (<ipython-input-3-c12bd3f3d540>,Line 8 "<built-in method strip of str object at 0x0000019DCFE31A50>"):Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=97.0.4692.71)

25/01/2022  scraping complete
 
------

In [8]:
#checking scraped Flight Price data
df=pd.read_csv("Flight.csv")
df

,Unnamed: 0,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Spicejet,25/01/2022,Goa,Ahmedabad,06:20,08:20,Non stop,"₹ 5,315"
2,1,Go First,25/01/2022,Goa,Ahmedabad,13:15,20:25,1 stop via Mumbai,"₹ 5,315"
3,2,Go First,25/01/2022,Goa,Ahmedabad,14:00,20:55,1 stop via Bengaluru,"₹ 5,315"
4,3,Go First,25/01/2022,Goa,Ahmedabad,23:55,06:40,1 stop via Mumbai,"₹ 5,315"
...,...,...,...,...,...,...,...,...,...
1529,15,Air India,25/01/2022,Bengaluru,Mumbai,06:45,08:35,Non stop,"₹ 5,179"
1530,16,Air India,25/01/2022,Bengaluru,Mumbai,19:10,21:05,Non stop,"₹ 5,179"
1531,17,Vistara,25/01/2022,Bengaluru,Mumbai,09:30,11:25,Non stop,"₹ 5,180"
1532,18,Vistara,25/01/2022,Bengaluru,Mumbai,19:55,21:40,Non stop,"₹ 5,757"


In [9]:
#saving Flight Price data as csv file
df.to_csv("flight_price.csv")